In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import calendar
import datetime as dt
import datetime
import pickle
import azureml.train.automl
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from math import sqrt
from lifelines.utils import k_fold_cross_validation
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns
from sklearn.datasets.samples_generator import (make_blobs,
                                                make_circles,
                                                make_moons)
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline
sns.set_context('notebook')
plt.style.use('fivethirtyeight')
from warnings import filterwarnings
filterwarnings('ignore')

Duplicate key in file '/Users/at185217/.matplotlib/matplotlibrc' line #2.


In [2]:
df = pd.read_csv('EDL_test_data.csv')

In [3]:
df.shape

(683294, 95)

In [4]:
df.columns = df.columns.str.replace(r'test_data_t2p.', '')
df['customer_nbr'] = df['customer_nbr'].astype(str)
df['invoice_nbr'] = df['invoice_nbr'].astype(str)
df["ratio_closed_0"] = df["closed_bucket_0"]/df["count_paid_invoices"]
df["ratio_closed_1"] = df["closed_bucket_1"]/df["count_paid_invoices"]
df["ratio_closed_2"] = df["closed_bucket_2"]/df["count_paid_invoices"]
df["ratio_closed_3"] = df["closed_bucket_3"]/df["count_paid_invoices"]
df["ratio_closed_4"] = df["closed_bucket_4"]/df["count_paid_invoices"]
df = df.loc[df['customer_segment'] == 1]
print(df.shape)
print(df.customer_segment.value_counts())

(162593, 100)
1    162593
Name: customer_segment, dtype: int64


In [5]:
df.closed_bucket_1.head()

0    5
1    6
2    5
3    5
4    5
Name: closed_bucket_1, dtype: int64

In [6]:
df.ratio_closed_1.head()

0   0.227
1   0.273
2   0.156
3   0.156
4   0.156
Name: ratio_closed_1, dtype: float64

In [7]:
# Load Model
filename = 'Survival_Model_EDL_Data.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [8]:
test = df[['country_code','invoice_base_amount','commited_days', 'invoice_category',  'financial_banking_cfi',
            'financial_banking_major', 'financial_channel', 'financial_other_financial_serv','hospitality_channel',
            'hospitality_eating_drinking_qs','hospitality_eating_drinking_ts','hospitality_sports_recreation','retail_channel',
            'retail_cstore_and_petroleum','retail_distribution_wholesale','retail_drug','retail_food_and_mass',
            'retail_public_sector','retail_specialty_retail','t_and_t_technology','tlg_channel',
            'tlg_gaming','tlg_lodging','tlg_travel',
            'month_num_created','day_of_week_created','day_of_month_due','month_num_due','day_of_week_due',
            'ratio_late_paid_amount',
            'ratio_closed_0', 'ratio_closed_1', 'ratio_closed_2', 'ratio_closed_3', 'ratio_closed_4', 
            'hw','sw','swm','hwm','ps','ts','other_category']]

In [9]:
test2=test.fillna(0)

In [10]:
test2.shape

(162593, 42)

In [11]:
#Predicting from the trained model
y_pred=loaded_model.predict_median(test2)

In [12]:
df['Pred_T2P'] = y_pred
df['Pred_T2P']=df['Pred_T2P'].replace(np.inf,999.0)
df['Predicted_Date'] = pd.to_datetime(df['invoice_date']) + pd.to_timedelta(df['Pred_T2P'], unit='d')
feb=df.loc[(df['Predicted_Date']>='2020-02-01') & (df['Predicted_Date']<='2020-02-29')]
print(feb.invoice_base_amount.sum())

40682360.32


In [14]:
df.loc[df['customer_nbr'] == '83263']

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date


In [13]:
df.head(20)

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date
0,10000718,1,6501327472,405.380,30,1,0,0,30,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-12,2020-01-12,2020-02-11,2020-02-11,12,1,6.000,11,2,1.000,12,1,6.000,0,0,1,1,0,0.000,22,5,0.230,5759.940,1530.390,0.270,1.600,0.000,0,0.000,0.000,0.000,0.000,0.000,17,5,0,0,0,0,16,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.773,0.227,0.000,0.000,0.000,30.000,2020-02-11
1,10001180,1,6501337072,440.540,30,1,0,0,30,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-12,2020-01-12,2020-02-11,2020-02-11,12,1,6.000,11,2,1.000,12,1,6.000,0,0,1,1,0,0.000,22,6,0.270,13127.040,3036.080,0.230,1.500,0.000,0,0.000,0.000,0.000,0.000,0.000,16,6,0,0,0,0,24,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.727,0.273,0.000,0.000,0.000,30.000,2020-02-11
2,10001749,1,200345741,135.000,30,1,0,4,983876,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-03-28,2019-03-28,2019-04-27,4712-12-31,28,3,3.000,27,4,5.000,28,3,3.000,1,1,0,0,6,0.190,32,26,0.810,7478.460,5944.250,0.790,133.080,19.000,19,1.000,5611.670,5611.670,1.000,221.260,6,5,1,0,20,19,52,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.188,0.156,0.031,0.000,0.625,270.000,2019-12-23
3,10001749,1,6006838405,178.000,30,1,0,4,983980,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018-12-14,2018-12-14,2019-01-13,4712-12-31,14,12,4.000,13,1,6.000,14,12,4.000,0,0,1,1,6,0.190,32,26,0.810,7478.460,5944.250,0.790,133.080,19.000,19,1.000,5611.670,5611.670,1.000,221.260,6,5,1,0,20,19,8,0.250,0.000,0.000,0.000,0.000,0.000,0.750,0,1,0.188,0.156,0.031,0.000,0.625,341.000,2019-11-20
4,10001749,1,6006858312,1525.000,30,1,0,4,983944,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-19,2019-01-19,2019-02-18,4712-12-31,19,1,5.000,18,2,0.000,19,1,5.000,0,0,0,0,6,0.190,32,26,0.810,7478.460,5944.250,0.790,133.080,19.000,19,1.000,5611.670,5611.670,1.000,221.260,6,5,1,0,20,19,64,0.230,0.000,0.000,0.000,0.000,0.020,0.750,0,1,0.188,0.156,0.031,0.000,0.625,153.000,2019-06-21
5,10001749,1,8081960,451.000,30,1,0,4,983863,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-04-10,2019-04-10,2019-05-10,4712-12-31,10,4,2.000,10,5,4.000,10,4,2.0

In [15]:
df.loc[df['invoice_nbr'].isin(['100369592','100371067','100394738','100395419','100413164'])]

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date
254,10097332,1,100395419,53.000,30,1,0,4,983963,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018-12-31,2018-12-31,2019-01-30,4712-12-31,31,12,0.000,30,1,2.000,31,12,0.000,1,1,0,0,0,0.000,14,6,0.430,6366.000,3155.000,0.500,1.330,2.000,2,1.000,211.000,211.000,1.000,338.000,8,6,0,0,0,2,16,0.250,0.000,0.000,0.000,0.000,0.000,0.750,0,1,0.571,0.429,0.000,0.000,0.000,30.000,2019-01-30
184127,7392910,1,100369592,6970.040,30,1,0,4,984130,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018-07-17,2018-07-17,2018-08-16,4712-12-31,17,7,1.000,16,8,3.000,17,7,1.000,0,0,0,0,8,0.040,221,213,0.960,1224330.640,1215704.000,0.990,116.700,505.000,505,1.000,6643683.820,6643683.820,1.000,189.340,8,58,46,16,93,437,156,0.120,0.010,0.000,0.000,0.000,0.000,0.870,0,1,0.036,0.262,0.208,0.072,0.421,279.000,2019-04-22
186442,8140888,1,100371067,153.280,30,1,0,4,984122,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018-07-25,2018-07-25,2018-08-24,4712-12-31,25,7,2.000,24,8,4.000,25,7,2.000,0,0,0,0,2,0.170,12,10,0.830,12634.310,8408.790,0.670,139.000,2.000,2,1.000,356.800,356.800,1.000,529.500,2,4,1,0,5,2,14,0.140,0.000,0.000,0.000,0.000,0.000,0.860,0,1,0.167,0.333,0.083,0.000,0.417,126.000,2018-11-28
338591,7393019,1,100413164,2498.000,30,1,0,4,983860,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2019-04-13,2019-04-13,2019-05-13,4712-12-31,13,4,5.000,13,5,0.000,13,4,5.000,0,0,1,1,29,0.310,94,65,0.690,729792.890,594555.350,0.810,117.090,9.000,9,1.000,69342.500,69342.500,1.000,201.670,29,42,1,1,21,9,12,0.250,0.000,0.000,0.000,0.000,0.000,0.750,0,1,0.309,0.447,0.011,0.011,0.223,47.000,2019-05-30
539663,10235088,1,100394738,681.610,30,1,0,4,983966,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018-12-28,2018-12-28,2019-01-27,4712-12-31,28,12,4.000,27,1,6.000,28,12,4.000,0,1,0,0,9,0.690,13,4,0.310,47334.230,23750.000,0.500,171.000,15.000,15,1.000,3220.840,3220.840,1.000,180.130,9,0,0,0,4,13,16,0.500,0.000,0.000,0.000,0.000,0.000,0.500,0,1,0.692,0.000,0.000,0.000,0.308,46.000,2019-02-12


In [16]:
df.invoice_nbr.nunique()

162593

In [17]:
surv = loaded_model.predict_survival_function(df)

In [18]:
surv.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19,20,21,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,76,77,78,79,82,83,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,111,112,113,114,115,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,192,193,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218,221,222,223,224,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,243,245,246,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,267,268,270,271,275,276,277,278,279,280,281,282,283,285,286,287,288,289,290,291,...,683007,683008,683009,683010,683011,683012,683013,683014,683015,683016,683017,683018,683019,683020,683021,683022,683023,683024,683025,683026,683027,683028,683029,683030,683031,683032,683033,683034,683035,683036,683037,683038,683039,683040,683042,683043,683044,683045,683046,683047,683048,683050,683051,683052,683053,683054,683055,683056,683057,683058,683059,683060,683061,683062,683063,683064,683065,683067,683068,683069,683070,683071,683072,683073,683074,683075,683077,683078,683079,683080,683081,683082,683083,683084,683085,683086,683087,683088,683089,683090,683091,683092,683093,683094,683095,683096,683098,683099,683100,683101,683103,683104,683105,683106,683108,683109,683110,683111,683112,683114,683115,683116,683117,683118,683119,683120,683121,683122,683123,683124,683128,683129,683132,683136,683137,683139,683140,683143,683144,683145,683146,683147,683148,683149,683150,683151,683152,683154,683156,683157,683158,683159,683160,683161,683162,683163,683164,683165,683166,683167,683168,683169,683170,683171,683172,683173,683174,683175,683176,683177,683178,683179,683180,683183,683184,683185,683188,683189,683190,683191,683192,683193,683194,683195,683196,683197,683198,683199,683200,683201,683202,683204,683207,683208,683209,683210,683211,683213,683214,683215,683216,683217,683218,683219,683220,683221,683223,683224,683225,683226,683227,683229,683230,683231,683232,683233,683234,683235,683236,683237,683239,683240,683241,683242,683243,683244,683245,683246,683247,683248,683250,683251,683252,683253,683254,683255,683256,683257,683258,683259,683260,683261,683262,683263,683264,683265,683266,683267,683269,683270,683271,683272,683273,683274,683275,683276,683277,683278,683279,683280,683281,683282,683283,683284,683285,683286,683287,683289,683290,683291
0.000,0.996,0.996,0.999,0.999,0.999,0.999,0.996,1.000,1.000,1.000,1.000,1.000,1.000,0.999,0.997,0.999,0.999,0.995,0.996,0.996,0.999,0.999,0.999,0.996,0.998,0.995,1.000,0.997,0.997,0.995,1.000,0.994,0.997,0.997,0.998,0.997,1.000,0.997,0.998,0.999,0.998,0.999,0.996,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.996,0.996,0.996,0.993,0.998,0.996,1.000,0.996,0.994,0.996,0.999,0.996,0.996,0.997,0.998,0.998,1.000,1.000,0.993,0.997,0.998,0.996,0.997,0.998,0.996,0.997,0.996,0.997,0.998,0.997,1.000,0.995,0.996,0.997,0.995,0.996,0.996,0.997,0.999,0.998,0.997,0.997,0.996,0.996,0.998,0.998,0.997,0.996,0.996,0.999,0.997,0.997,0.996,0.999,0.995,0.994,0.995,0.995,0.995,0.996,0.998,0.998,0.998,0.999,0.998,0.998,0.996,0.999,0.997,0.997,0.997,0.997,0.996,0.997,0.999,0.996,0.996,0.998,0.997,0.996,1.000,0.998,0.997,0.996,0.999,0.997,0.997,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.997,0.997,0.997,0.997,0.998,0.998,1.000,0.997,0.996,0.996,0.999,0.997,0.994,0.994,0.997,1.000,0.997,0.996,0.999,0.998,0.998,0.998,0.996,0.999,0.998,0.996,0.997,0.998,0.996,0.997,0.998,0.998,0.999,0.996,0.999,0.999,0.995,0.995,0.996,0.996,0.996,1.000,0.995,0.998,0.997,0.996,0.998,0.998,0.996,0.998,0.999,0.999,0.999,0.999,0.999,0.999,0.998,0.998,0.998,0.996,0.996,1.000,0.996,0.996,0.997,0.996,0.998,0.997,0.996,1.000,1.000,0.

In [19]:
surv.reset_index(inplace=True)
surv.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19,20,21,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,76,77,78,79,82,83,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,111,112,113,114,115,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,192,193,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218,221,222,223,224,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,243,245,246,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,267,268,270,271,275,276,277,278,279,280,281,282,283,285,286,287,288,289,290,...,683007,683008,683009,683010,683011,683012,683013,683014,683015,683016,683017,683018,683019,683020,683021,683022,683023,683024,683025,683026,683027,683028,683029,683030,683031,683032,683033,683034,683035,683036,683037,683038,683039,683040,683042,683043,683044,683045,683046,683047,683048,683050,683051,683052,683053,683054,683055,683056,683057,683058,683059,683060,683061,683062,683063,683064,683065,683067,683068,683069,683070,683071,683072,683073,683074,683075,683077,683078,683079,683080,683081,683082,683083,683084,683085,683086,683087,683088,683089,683090,683091,683092,683093,683094,683095,683096,683098,683099,683100,683101,683103,683104,683105,683106,683108,683109,683110,683111,683112,683114,683115,683116,683117,683118,683119,683120,683121,683122,683123,683124,683128,683129,683132,683136,683137,683139,683140,683143,683144,683145,683146,683147,683148,683149,683150,683151,683152,683154,683156,683157,683158,683159,683160,683161,683162,683163,683164,683165,683166,683167,683168,683169,683170,683171,683172,683173,683174,683175,683176,683177,683178,683179,683180,683183,683184,683185,683188,683189,683190,683191,683192,683193,683194,683195,683196,683197,683198,683199,683200,683201,683202,683204,683207,683208,683209,683210,683211,683213,683214,683215,683216,683217,683218,683219,683220,683221,683223,683224,683225,683226,683227,683229,683230,683231,683232,683233,683234,683235,683236,683237,683239,683240,683241,683242,683243,683244,683245,683246,683247,683248,683250,683251,683252,683253,683254,683255,683256,683257,683258,683259,683260,683261,683262,683263,683264,683265,683266,683267,683269,683270,683271,683272,683273,683274,683275,683276,683277,683278,683279,683280,683281,683282,683283,683284,683285,683286,683287,683289,683290,683291
0,0.000,0.996,0.996,0.999,0.999,0.999,0.999,0.996,1.000,1.000,1.000,1.000,1.000,1.000,0.999,0.997,0.999,0.999,0.995,0.996,0.996,0.999,0.999,0.999,0.996,0.998,0.995,1.000,0.997,0.997,0.995,1.000,0.994,0.997,0.997,0.998,0.997,1.000,0.997,0.998,0.999,0.998,0.999,0.996,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.996,0.996,0.996,0.993,0.998,0.996,1.000,0.996,0.994,0.996,0.999,0.996,0.996,0.997,0.998,0.998,1.000,1.000,0.993,0.997,0.998,0.996,0.997,0.998,0.996,0.997,0.996,0.997,0.998,0.997,1.000,0.995,0.996,0.997,0.995,0.996,0.996,0.997,0.999,0.998,0.997,0.997,0.996,0.996,0.998,0.998,0.997,0.996,0.996,0.999,0.997,0.997,0.996,0.999,0.995,0.994,0.995,0.995,0.995,0.996,0.998,0.998,0.998,0.999,0.998,0.998,0.996,0.999,0.997,0.997,0.997,0.997,0.996,0.997,0.999,0.996,0.996,0.998,0.997,0.996,1.000,0.998,0.997,0.996,0.999,0.997,0.997,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.997,0.997,0.997,0.997,0.998,0.998,1.000,0.997,0.996,0.996,0.999,0.997,0.994,0.994,0.997,1.000,0.997,0.996,0.999,0.998,0.998,0.998,0.996,0.999,0.998,0.996,0.997,0.998,0.996,0.997,0.998,0.998,0.999,0.996,0.999,0.999,0.995,0.995,0.996,0.996,0.996,1.000,0.995,0.998,0.997,0.996,0.998,0.998,0.996,0.998,0.999,0.999,0.999,0.999,0.999,0.999,0.998,0.998,0.998,0.996,0.996,1.000,0.996,0.996,0.997,0.996,0.998,0.997,0.996,1.000,1.00

In [51]:
sample = df.sample(n =100, random_state = 0)
sample.shape

(100, 102)

In [54]:
dataframe = pd.DataFrame(columns=['invoice_nbr','number_of_days','survival_prob'])
for index,row in sample.iterrows():
    a=loaded_model.predict_survival_function(row)
    a=a.iloc[:200,:]
    a.reset_index(inplace=True)
    a['invoice_nbr'] = row['invoice_nbr']
    a.columns = ['number_of_days', 'survival_prob', 'invoice_nbr']
    dataframe = dataframe.append(a)

In [55]:
dataframe.shape

(20000, 3)

In [56]:
dataframe.head()

,invoice_nbr,number_of_days,survival_prob
0,6500704336,0.000,0.998
1,6500704336,1.000,0.997
2,6500704336,2.000,0.996
3,6500704336,3.000,0.953
4,6500704336,4.000,0.934


In [57]:
dataframe.iloc[195:205,:]

,invoice_nbr,number_of_days,survival_prob
195,6500704336,195.000,0.155
196,6500704336,196.000,0.155
197,6500704336,197.000,0.154
198,6500704336,198.000,0.154
199,6500704336,199.000,0.154
0,6501298825,0.000,0.999
1,6501298825,1.000,0.999
2,6501298825,2.000,0.998
3,6501298825,3.000,0.979
4,6501298825,4.000,0.970


In [63]:
t = df.loc[df['invoice_nbr'] =='6501298825']
tf = loaded_model.predict_survival_function(t)

In [68]:
sample.to_csv('DataPrepSample.csv', index=False)
dataframe.to_csv('DataPrepProbability.csv', index=False)

In [78]:
a = sample.groupby('customer_nbr')['invoice_base_amount'].sum()
b = pd.DataFrame(a)
b.reset_index(inplace=True)
b.sort_values(by=['invoice_base_amount'], ascending=False)
b.rename(columns={'invoice_base_amount':'Total_AR'}, inplace=True)
b.head()

,customer_nbr,Total_AR
0,10031138,1157.630
1,10055076,189.320
2,10061676,139.000
3,10137074,504.380
4,10230473,101.610


In [79]:
sample2 = sample.loc[(sample['Predicted_Date']>='2020-02-01') & (sample['Predicted_Date']<='2020-02-29')]
x = sample2.groupby('customer_nbr')['invoice_base_amount'].sum()
y = pd.DataFrame(x)
y.reset_index(inplace=True)
y.sort_values(by=['invoice_base_amount'], ascending=False)
y.rename(columns={'invoice_base_amount':'Forecast'}, inplace=True)
y.head()

,customer_nbr,Forecast
0,10299094,30.000
1,3040989,16.710
2,6675905,12.410
3,7392984,22.500
4,7425938,731.500


In [80]:
z= b.merge(y,how='outer',on='customer_nbr')
z.sort_values(by=['Total_AR'], ascending=False)

,customer_nbr,Total_AR,Forecast
34,8140409,261833.100,nan
54,9437369,15719.100,nan
18,5001355,10547.580,nan
20,7114,10100.830,nan
8,155985,4202.900,nan
69,9805752,3822.710,nan
35,8140696,2670.000,nan
68,9757937,1487.000,nan
53,9427298,1403.990,nan
0,10031138,1157.630,nan


In [81]:
z.Total_AR.sum()

326060.53

In [82]:
z.Forecast.sum()

2718.84

In [83]:
z.loc[z['customer_nbr']=='6675905']

,customer_nbr,Total_AR,Forecast
19,6675905,36.670,12.410


In [85]:
df.loc[df['customer_nbr']=='7114']

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date
30047,7114,1,2827727484,921.000,30,1,0,4,983918,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-02-14,2019-02-14,2019-03-16,4712-12-31,14,2,3.000,16,3,5.000,14,2,3.000,0,0,0,0,61,0.490,124,63,0.510,2242620.290,637569.640,0.280,234.370,23.000,23,1.000,206140.860,206140.860,1.000,266.650,61,3,0,0,60,22,2,0.000,0.000,0.000,0.000,0.500,0.000,0.500,0,1,0.492,0.024,0.000,0.000,0.484,47.000,2019-04-02
30048,7114,1,2829341520,10546.630,30,1,0,4,983588,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-10,2020-01-10,2020-02-09,4712-12-31,10,1,4.000,9,2,6.000,10,1,4.000,0,0,1,1,61,0.490,124,63,0.510,2242620.290,637569.640,0.280,234.370,23.000,23,1.000,206140.860,206140.860,1.000,266.650,61,3,0,0,60,22,34,0.000,0.000,0.410,0.090,0.000,0.000,0.500,0,1,0.492,0.024,0.000,0.000,0.484,61.000,2020-03-11
30049,7114,1,2829341526,10546.630,30,1,0,4,983588,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-10,2020-01-10,2020-02-09,4712-12-31,10,1,4.000,9,2,6.000,10,1,4.000,0,0,1,1,61,0.490,124,63,0.510,2242620.290,637569.640,0.280,234.370,23.000,23,1.000,206140.860,206140.860,1.000,266.650,61,3,0,0,60,22,34,0.000,0.000,0.410,0.090,0.000,0.000,0.500,0,1,0.492,0.024,0.000,0.000,0.484,61.000,2020-03-11
68646,7114,1,2829340698,11824.360,30,1,0,4,983588,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-10,2020-01-10,2020-02-09,4712-12-31,10,1,4.000,9,2,6.000,10,1,4.000,0,0,1,1,61,0.490,124,63,0.510,2242620.290,637569.640,0.280,234.370,23.000,23,1.000,206140.860,206140.860,1.000,266.650,61,3,0,0,60,22,44,0.000,0.000,0.410,0.090,0.000,0.000,0.500,0,1,0.492,0.024,0.000,0.000,0.484,62.000,2020-03-12
68647,7114,1,2829341523,10575.580,30,1,0,4,983588,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-10,2020-01-10,2020-02-09,4712-12-31,10,1,4.000,9,2,6.000,10,1,4.000,0,0,1,1,61,0.490,124,63,0.510,2242620.290,637569.640,0.280,234.370,23.000,23,1.000,206140.860,206140.860,1.000,266.650,61,3,0,0,60,22,30,0.000,0.000,0.270,0.230,0.000,0.000,0.500,0,1,0.492,0.024,0.000,0.000,0.484,54.000,2020-03-04
106889,7114,1,2829318210,10100.830,30,1,0,4,983601,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [91]:
a = df.groupby('customer_nbr')['invoice_base_amount'].sum()
b = pd.DataFrame(a)
b.reset_index(inplace=True)
b.sort_values(by=['invoice_base_amount'], ascending=False)
b.rename(columns={'invoice_base_amount':'Total_AR'}, inplace=True)
b.head()

,customer_nbr,Total_AR
0,10000109,515.700
1,10000210,341.250
2,10000230,50.000
3,10000260,53.300
4,10000716,320.000


In [93]:
sample2 = df.loc[(df['Predicted_Date']>='2020-02-01') & (df['Predicted_Date']<='2020-02-29')]
x = sample2.groupby('customer_nbr')['invoice_base_amount'].sum()
y = pd.DataFrame(x)
y.reset_index(inplace=True)
y.sort_values(by=['invoice_base_amount'], ascending=False)
y.rename(columns={'invoice_base_amount':'Forecast'}, inplace=True)
y.head()

,customer_nbr,Forecast
0,10000109,515.700
1,10000210,341.250
2,10000230,50.000
3,10000260,53.300
4,10000716,320.000


In [94]:
z= b.merge(y,how='outer',on='customer_nbr')
z.sort_values(by=['Total_AR'], ascending=False)

,customer_nbr,Total_AR,Forecast
21999,9915862,53832024.800,nan
4606,7392910,7155874.160,386392.940
20951,9805752,4591221.790,573244.190
4290,5169760,2512195.000,215138.570
9286,8140680,2137934.510,110810.940
8521,8038346,2094551.060,244679.170
4063,37548,1533494.440,1030274.380
4170,44679,1516961.890,49961.090
4173,4482354,1471835.170,nan
2151,10205343,1438059.340,nan


In [95]:
z.isna().sum()

customer_nbr       0
Total_AR           0
Forecast        6541
dtype: int64

In [96]:
z.shape

(22905, 3)

In [99]:
df.to_csv('DataPrep_EntireData.csv', index=False)

In [101]:
z.loc[z['customer_nbr'] == '5001355']

,customer_nbr,Total_AR,Forecast
4219,5001355,725886.460,15932.180


In [102]:
z.isna().sum()

customer_nbr       0
Total_AR           0
Forecast        6541
dtype: int64

In [104]:
df.loc[df['invoice_nbr'] == '7724774']

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date
566393,5001633CA,0,7724774,13836.780,30,1,0,4,983652,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-11-07,2019-11-07,2019-12-07,4712-12-31,7,11,3.000,7,12,5.000,7,11,3.000,0,0,1,1,1,0.100,10,9,0.900,17494.320,15378.520,0.880,52.670,3.000,3,1.000,30636.420,30636.420,1.000,22.330,1,2,4,2,1,0,141,0.310,0.000,0.000,0.000,0.000,0.020,0.670,0,1,0.100,0.200,0.400,0.200,0.100,101.000,2020-02-16


In [105]:
df.invoice_base_amount.describe()

count     162593.000
mean        1642.878
std       133673.820
min            0.010
25%           54.000
50%          158.000
75%          439.000
max     53768484.000
Name: invoice_base_amount, dtype: float64

In [106]:
df.customer_segment.value_counts()

1    162593
Name: customer_segment, dtype: int64